In [1]:
import requests
from threading import Thread
from lxml import etree
import json

In [2]:
url = 'https://www.pearvideo.com/'

In [3]:
def get_info():
    headers={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    res = etree.HTML(response.text)
    id_list = res.xpath('//a[@class="vervideo-lilink actplay"]/@href')
    id_list = [id_list[i][6:] for i in range(len(id_list))] #获取视频ID
    name_list = res.xpath('//*[@class="vervideo-name"]/text()') + res.xpath('//*[@class="vervideo-title"]/text()') #获取视频名字
    return id_list, name_list

In [7]:
def get_url(id_list, name_list):
    for i in range(5):
        new_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36', 
                     'Referer': f'https://www.pearvideo.com/{id_list[i]}'}
        response = requests.get(url=f'https://www.pearvideo.com/videoStatus.jsp?contId={id_list[i]}', headers=new_headers).json()
        video_url = response["videoInfo"]["videos"]["srcUrl"]
        video_url = video_url.replace(video_url.split("/")[-1].split("-")[0], "cont-" + id_list[i])
        headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
        }
        video = requests.get(url=video_url, headers=headers).content 
        with open(f"./11_多线程_梨视频/{name_list[i]}.mp4", "wb") as f: 
            print(f"正在下载视频 {name_list[i]} ...") 
            f.write(video) 
            print(f"视频 {name_list[i]} 下载完成！") 

In [10]:
t = Thread(target=get_url, args=(get_info()[0], get_info()[1]))
t.start()

In [11]:
t

<Thread(Thread-6, started 23840)>

正在下载视频 90岁大爷贴身照顾瘫痪妻子，还帮助养老院其他老人 ...
视频 90岁大爷贴身照顾瘫痪妻子，还帮助养老院其他老人 下载完成！
正在下载视频 80后女汉子打理千亩农田：无人机除草，全机械作业 ...
视频 80后女汉子打理千亩农田：无人机除草，全机械作业 下载完成！
正在下载视频 宝藏老师｜物理老师会“魔术”，与学生手拉手就能点亮灯泡 ...
视频 宝藏老师｜物理老师会“魔术”，与学生手拉手就能点亮灯泡 下载完成！
正在下载视频 重庆2岁家猫成职业车模，出场费最高上万，服装比小姐姐还多 ...
视频 重庆2岁家猫成职业车模，出场费最高上万，服装比小姐姐还多 下载完成！
正在下载视频 魔幻的重庆深不可测，地下106米藏着全国最深地铁站 ...
视频 魔幻的重庆深不可测，地下106米藏着全国最深地铁站 下载完成！
